In [ ]:
# FAISS : 대규모 데이터셋에서 고차원 벡터의 유사도 검색을 빠르게 수행할 수 있도록 설계된 라이브러리
# pip intatll faiss-cpu / faiss-gpu

# 밀집검색 (dense retriever) :
# 문서와 쿼리를 고차원 밀집 벡터 형태로 표현하여 검색을 수행하는 방법 (p. 244)
# 희소 검색과 달리, 단어의 존재 여부나 빈도만이 아닌 단어의 의미와 문맥을 고려하여 검색을 수행

# 밀집검색의 작동 과정
# 문서 변환 -> 질문 처리 -> 유사도 계산 -> 랭킹 -> 결과 반환

In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

file_path = (
    "C:\Dev\LLM_LangChain_RAG_250521\data_stock\Samsung_2024.pdf"
)

loader = PyPDFLoader(file_path)

doc_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

docs = loader.load_and_split(doc_splitter)

In [8]:
# OpenAI 임베딩 모델

# from langchain_openai.embeddings import OpenAIEmbeddings

# embedding = OpenAIEmbeddings(model="text-embedding-3-large")    # API Key가 없으므로 HuggingFace로 대체

from langchain.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',    # 다국어 모델
    # model_name='jhgan/ko-sroberta-multitask',  # 한국어 모델 - 에러 발생 (250603)
    # model_name = 'BAAI/bge-m3',                # 에러 발생 (250603)
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)

embeddings_model

C:\Users\wonta\AppData\Local\Temp\ipykernel_11808\2953665233.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, multi_process=False, show_progress=False)

In [11]:
# FAISS 라이브러리 임포트

from langchain_community.vectorstores import FAISS

# FAISS DB 생성 후 저장

faiss_store = FAISS.from_documents(docs, embeddings_model)
faiss_store.save_local("./content/DB")

In [13]:
# 저장된 DB 경로 지정 후, DB 로드

persist_folder = "./content/DB"
vectordb = FAISS.load_local(persist_folder, embeddings=embeddings_model, allow_dangerous_deserialization=True)

In [17]:
# FAISS Retriever 생성

faiss_retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [27]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain_openai import ChatOpenAI
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 관련있는 문서 수집 후, 챗GPT로 최종 답변 수행
# 여기서는 Local(LM Tools)에서 실행

llm_lmtool = ChatOpenAI(
    base_url="http://192.168.25.5:11434/v1",
    api_key="lm-studio",
    model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
    temperature=0.2,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()], # 스트림 출력 콜백
)

qa_chain = RetrievalQA.from_chain_type(
    llm = llm_lmtool,
    chain_type = "stuff",
    retriever=faiss_retriever,
    return_source_documents=True   # 답변에 사용된 source document를 보여주도록 설정
)

In [28]:
qa_chain.invoke("이 회사가 발행한 주식의 총 발행량을 알려줘.")

회사의 정관에 의한 발행할 주식의 총수는 25,000,000,000주입니다.

{'query': '이 회사가 발행한 주식의 총 발행량을 알려줘.',
 'result': '회사의 정관에 의한 발행할 주식의 총수는 25,000,000,000주입니다.',
 'source_documents': [Document(id='747ab3b6-2987-4662-b86d-f1c1ec283440', metadata={'producer': 'iText® 5.4.0 ©2000-2012 1T3XT BVBA (AGPL-version)', 'creator': 'PyPDF', 'creationdate': '2025-03-11T17:01:52+09:00', 'moddate': '2025-03-11T17:01:52+09:00', 'source': 'C:\\Dev\\LLM_LangChain_RAG_250521\\data_stock\\Samsung_2024.pdf', 'total_pages': 527, 'page': 131, 'page_label': '129'}, page_content='회사의 정관에 의한 발\n행할 주식의 총수(주) \u3000 \u3000 25,000,000,000\n주당 액면가액(원) \u3000 \u3000 100\n발행주식 액면 총액(백\n만원) 596,978 82,289 679,267\n납입자본금(백만원) \u3000 \u3000 897,514\n자본금에 대한 설명 \u3000 \u3000\n보고기간종료일 현재 회\n사의 정관에 의한 발행\n할 주식의 총수는 250억\n주(1주의 액면금액:\n100원)이며, 회사에는\n보통주 외 비누적적 우\n선주가 있습니다. 이 우\n선주는 의결권이 없고\n액면금액을 기준으로 보\n통주의 배당보다 연\n1%의 금전배당을 추가\n로 받을 수 있습니다. 보\n고기간종료일 현재 회사\n가 발행한 보통주 및 우\n선주의 수(소각 주식수\n제외)는 각각\n5,969,782,550주와\n822,886,700주이며, 유\n통주식수는 자기주식 취\n득으로 인하여 발행주식\n의 총수와 차이가 있습\n니다. 한편, 발행주식의\n

In [29]:
qa_chain.invoke("이 회사의 2025년 계획을 알려줘.")

제공된 정보에서 2025년 계획을 찾을 수 있는 부분은 다음과 같습니다.

* 조기입증감사(연중감사) : 2024.03.06 ~ 2024.12.29 (이러한 일정은 연중 감사를 의미하므로, 2025년에도 연중 감사가 진행될 것으로 추정됩니다.)
* 내부회계관리제도 감사 : 2024.04.03 ~ 2025.01.31
* 조기입증감사 절차 업데이트 및 재무제표 감사 : 2025.01.02 ~ 2025.02.18

이러한 일정은 회사의 내부 감사를 의미하며, 2025년의 계획을 자세히 알 수는 없습니다.

{'query': '이 회사의 2025년 계획을 알려줘.',
 'result': '제공된 정보에서 2025년 계획을 찾을 수 있는 부분은 다음과 같습니다.\n\n* 조기입증감사(연중감사) : 2024.03.06 ~ 2024.12.29 (이러한 일정은 연중 감사를 의미하므로, 2025년에도 연중 감사가 진행될 것으로 추정됩니다.)\n* 내부회계관리제도 감사 : 2024.04.03 ~ 2025.01.31\n* 조기입증감사 절차 업데이트 및 재무제표 감사 : 2025.01.02 ~ 2025.02.18\n\n이러한 일정은 회사의 내부 감사를 의미하며, 2025년의 계획을 자세히 알 수는 없습니다.',
 'source_documents': [Document(id='d18b3605-1f1b-455b-8baf-8502ddb945c2', metadata={'producer': 'iText® 5.4.0 ©2000-2012 1T3XT BVBA (AGPL-version)', 'creator': 'PyPDF', 'creationdate': '2025-03-11T17:01:52+09:00', 'moddate': '2025-03-11T17:01:52+09:00', 'source': 'C:\\Dev\\LLM_LangChain_RAG_250521\\data_stock\\Samsung_2024.pdf', 'total_pages': 527, 'page': 312, 'page_label': '310'}, page_content='(제56기 일정)\n구  분 일  정\n제56기 1분기\n사전검토 2024.03.11 ~ 2024.03.29\n본검토 2024.04.08 ~ 2024.05.14\n제56기 2분기\n사전검토 2024.06.10 ~ 2024.06.27\n본검토 2024.07.08 ~ 2024.08.13\n제56기 3분기\n사전검토 2024.09.09 ~ 2024.09.27\n본검토 2024.10.08 ~ 2024.11.13\n시스템 및 자동 내부통제 감사 2024.04.03 ~ 2025.

In [30]:
qa_chain.invoke("이 회사의 2024년 경영 성과는 2023년 대비 어떠했는지 알려줘.")

제공된 정보에서 2024년의 경영 성과에 대한 직접적인 정보가 없습니다. 그러나, 배당수익률에 대한 정보를 제공하고 있습니다.

배당수익률은 2024년(제56기) 보통주 2.7%, 우선주 3.3%로, 2023년 대비 변동이 없는 것으로 나타납니다.

{'query': '이 회사의 2024년 경영 성과는 2023년 대비 어떠했는지 알려줘.',
 'result': '제공된 정보에서 2024년의 경영 성과에 대한 직접적인 정보가 없습니다. 그러나, 배당수익률에 대한 정보를 제공하고 있습니다.\n\n배당수익률은 2024년(제56기) 보통주 2.7%, 우선주 3.3%로, 2023년 대비 변동이 없는 것으로 나타납니다.',
 'source_documents': [Document(id='d18b3605-1f1b-455b-8baf-8502ddb945c2', metadata={'producer': 'iText® 5.4.0 ©2000-2012 1T3XT BVBA (AGPL-version)', 'creator': 'PyPDF', 'creationdate': '2025-03-11T17:01:52+09:00', 'moddate': '2025-03-11T17:01:52+09:00', 'source': 'C:\\Dev\\LLM_LangChain_RAG_250521\\data_stock\\Samsung_2024.pdf', 'total_pages': 527, 'page': 312, 'page_label': '310'}, page_content='(제56기 일정)\n구  분 일  정\n제56기 1분기\n사전검토 2024.03.11 ~ 2024.03.29\n본검토 2024.04.08 ~ 2024.05.14\n제56기 2분기\n사전검토 2024.06.10 ~ 2024.06.27\n본검토 2024.07.08 ~ 2024.08.13\n제56기 3분기\n사전검토 2024.09.09 ~ 2024.09.27\n본검토 2024.10.08 ~ 2024.11.13\n시스템 및 자동 내부통제 감사 2024.04.03 ~ 2025.01.31\n조기입증감사(연중감사) 2024.03.06 ~ 2024.12.29\n내부회계관리제도 감사 2024.04.03 ~ 2025.01.31\n조기입증감사 절차 업데이트 및 재무제표 감사 2025.01.02 ~ 2025.02.1

In [31]:
qa_chain.invoke("이 회사의 2024년 경영 성과를 200자 내외로 요약해줘.")

이 회사는 연결대상 종속기업의 총 수는 228개 사로 증가한 것을 보이고 있습니다. 주요 매출처로는 Apple, Deutsche Telekom, Hong Kong Techtronics, Supreme Electronics, Verizon 등이 있으며, 이들에 대한 매출 비중은 전체 매출액 대비 약 14% 수준입니다. 또한, 장기공급계약 수주거래는 없음을 알 수 있습니다.

{'query': '이 회사의 2024년 경영 성과를 200자 내외로 요약해줘.',
 'result': '이 회사는 연결대상 종속기업의 총 수는 228개 사로 증가한 것을 보이고 있습니다. 주요 매출처로는 Apple, Deutsche Telekom, Hong Kong Techtronics, Supreme Electronics, Verizon 등이 있으며, 이들에 대한 매출 비중은 전체 매출액 대비 약 14% 수준입니다. 또한, 장기공급계약 수주거래는 없음을 알 수 있습니다.',
 'source_documents': [Document(id='713f454f-f402-48a2-9f5a-7f456f6acdec', metadata={'producer': 'iText® 5.4.0 ©2000-2012 1T3XT BVBA (AGPL-version)', 'creator': 'PyPDF', 'creationdate': '2025-03-11T17:01:52+09:00', 'moddate': '2025-03-11T17:01:52+09:00', 'source': 'C:\\Dev\\LLM_LangChain_RAG_250521\\data_stock\\Samsung_2024.pdf', 'total_pages': 527, 'page': 9, 'page_label': '7'}, page_content="마. 연결대상 종속회사 현황(요약)\n \n기업회계기준서(K-IFRS) 제1110호 '연결재무제표'에 의한 지배회사(최상위)인 당사의 연결\n대상 종속기업은 2024년말 현재 228개사입니다. 전년말 대비 6개 기업이 증가하고 10개 기\n업이 감소하였습니다. \n(당기 연결대상회사의 변동내용) \n \n바. 중소기업 해당 여부 \n \n당사는「중소기업기본법」제2조에 의한 중소기업에 해당되지 않습니다.\n \n(단위 : 사)\n구분\n연결대상회사수 주요\n종속회사수기초 증가 감소 기말\n상장 - - - - -\n비상장 232 6 10 228 142\n합계 232 6 10 228 142\n※상